<a href="https://colab.research.google.com/github/hikmahealth/covid19countymap/blob/master/notebooks/UPDATED_Hikma_COVID_19_Response_County_Google_Sheet_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hikma COVID-19 Response - County Google Sheet Export

License: Apache 2.0

In [0]:
import copy
import io
import json
import urllib

from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
import pandas as pd


## Use census data to map county name to FIPS code

In [0]:
with urllib.request.urlopen("https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv") as infile:
  raw_data = infile.read()
# Deal with encoding issues in the Census CSV file.
replacements = {0xed: "í", 0xe1: "á", 0xf3: "ó", 0xf1: "ñ", 0xfc:"ü"}
for char, repl in replacements.items():
  raw_data = raw_data.replace(bytes([char]), repl.encode())
raw_census = pd.read_csv(io.BytesIO(raw_data))

In [0]:
census = raw_census[raw_census.COUNTY != 0].copy()

In [0]:
census["fips"] = census.STATE * 1000 + census.COUNTY
census.set_index("fips", inplace=True)
census["full_name"] = census.CTYNAME + ", " + census.STNAME
fips_code_mapping = {name: i for i, name in census.full_name.items()}

## Read the data from Google Sheets

In [0]:
auth.authenticate_user()
sheets_client = gspread.authorize(GoogleCredentials.get_application_default())
sheet = sheets_client.open_by_url("https://docs.google.com/spreadsheets/d/1LRqAKVYpa2uhQcwD8rragwuvU6gMLRzYupihsURC8Cg/edit#gid=519201207")

In [0]:
raw_policies = pd.DataFrame(sheet.sheet1.get_all_records())

## Clean up the county-level data

### Encode free-text column values

In [0]:
dimensions = ["school", "work", "shelter", "shelter_enforcement", "event", "testing", "transport"]

In [0]:
full_policies = raw_policies.rename(columns={
    "You are filling out the form for the following county:": "county_name",
    "Are schools closed in this county?": "school",
    "If yes, please provide the URL to information regarding school closures.": "school_url",
    "If yes, please provide the date of when this policy on school closures took or will take effect.": "school_date",
    "Are non-essential workplaces closed? Is there any language that suggests that those who do not provide essential services should work from home or stop working?": "work",
    "If yes, please provide the URL to information regarding non-essential workplace closures.": "work_url",
    "If yes, provide the date of when this policy took or will take effect.": "work_date",
    "Is social distancing or shelter-in-place being enforced with fines or penalties in this county? ": "shelter_enforcement",
    "If yes, please provide the URL to information regarding fine or penalty enforcement.": "shelter_enforcement_link",
    "If yes, please provide the date of when this fine or penalty enforcement took or will take effect.": "shelter_enforcement_date",
    "Is there a shelter-in-place or stay-at-home order in this county? For example, were people advised to maintain social distance or abstain from all physical interactions or non-essential trips whenever possible?": "shelter",
    "If yes, please provide the URL to information regarding the shelter-in-place or social distancing policy.": "shelter_url",
    "If yes, please provide the date of when this shelter-in-place or social distancing policy took or will take effect.": "shelter_date",
    "Is there any order that public events should be restricted in size or outright cancelled?": "event",
    "If yes, please provide the URL to information regarding public events restrictions.": "event_url",
    "If yes, please provide the date of when the cancellation of public events was or will be in effect.": "event_date",
    "Are there public facilities with COVID testing available in this county? ": "testing",
    "If yes, please provide the URL to information regarding testing facilities.": "testing_url",
    "If yes, please provide the date of when these facilities became available.": "testing_date",
    "Look for any information about whether this county's  buses, subways, light rail, etc. are being shut down. Is any public transit system shut down?": "transport",
    "If yes, please provide the URL to information regarding public transit.": "transport_url",
    "If yes, please provide the date of when public transit was or will be stopped.": "transport_date",
})
full_policies.pop("Are there travel restrictions in this county? For example, are people required to self-quarantine when arriving into this county?")
full_policies.pop("Are airports closed in this county? ")
full_policies["fips"] = full_policies.county_name.map(fips_code_mapping)
full_policies.set_index("fips", inplace=True)
full_policies["updated"] = pd.to_datetime(full_policies.pop("Timestamp"))
for col in dimensions:
  full_policies[col] = full_policies[col].map({"Yes": True, "No": False})
# Convert timestamp from ambiguous DD/MM/YYYY to ISO 8601 friendly YYYY-MM-DD
for col in dimensions:
  full_policies[col + "_date"] = pd.to_datetime(full_policies[col + "_date"]).dt.strftime("%Y-%m-%d")

In [9]:
full_policies.head()

,county_name,school,school_url,school_date,work,work_url,work_date,shelter_enforcement,shelter_enforcement_link,shelter_enforcement_date,shelter,shelter_url,shelter_date,event,event_url,event_date,testing,testing_url,testing_date,transport,transport_url,transport_date,updated
fips,,,,,,,,,,,,,,,,,,,,,,,
6073,"San Diego County, California",True,https://www.sandiegocounty.gov/content/sdc/hhs...,2020-03-16,True,https://www.sandiegocounty.gov/content/sdc/hhs...,2020-03-15,False,,NaN,True,https://www.sandiegocounty.gov/content/sdc/hhs...,2020-03-19,True,https://www.sandiegocounty.gov/content/sdc/hhs...,2020-03-16,True,https://health.ucsd.edu/coronavirus/Pages/defa...,NaN,True,https://www.sdmts.com/schedules-real-time/covi...,2020-04-12,2020-04-09 11:03:27
17031,"Cook County, Illinois",True,https://www.cookcountypublichealth.org/communi...,2020-03-17,True,https://www.cookcountypublichealth.org/communi...,2020-03-20,False,,NaN,True,https://www.cookcountypublichealth.org/communi...,2020-04-01,True,https://www.cookcountypublichealth.org/communi...,2020-04-01,False,,NaN,False,,NaN,2020-04-09 11:32:26
48201,"Harris County, Texas",True,https://publichealth.harriscountytx.gov/Resour...,2020-03-24,True,https://publichealth.harriscountytx.gov/Resour...,2020-03-24,True,https://www.readyharris.org/Stay-Home,2020-03-24,True,https://publichealth.harriscountytx.gov/Resour...,2020-03-20,True,https://www.readyharris.org/Stay-Home,2020-03-24,True,https://publichealth.harriscountytx.gov/Resour...,NaN,True,https://www.ridemetro.org/Pages/Coronavirus.aspx,2020-03-30,2020-04-09 18:08:51
4013,"Maricopa County, Arizona",True,https://azgovernor.gov/governor/news/2020/03/g...,2020-03-30,True,https://www.fox10phoenix.com/video/669286,2020-03-31,False,,NaN,False,https://www.theguardian.com/us-news/2020/mar/3...,NaN,False,https://www.maricopa.gov/Calendar.aspx,NaN,True,https://www.fox10phoenix.com/news/banner-healt...,2020-03-23,False,https://www.maricopa.gov/5307/Transportation-M...,NaN,2020-04-09 18:33:06
49043,"Summit County, Utah",True,http://www.pcschools.us/wp-content/uploads/202...,2020-03-16,True,https://www.summitcounty.org/DocumentCenter/Vi...,2020-03-27,True,https://www.summitcounty.org/DocumentCenter/Vi...,2020-03-27,True,https://www.summitcounty.org/DocumentCenter/Vi...,2020-03-27,True,https://www.summitcounty.org/DocumentCenter/Vi...,2020-03-27,True,https://kutv.com/news/local/curbside-coronavir...,2020-04-02,False,,NaN,2020-04-09 23:04:50


### Deduplicate rows.

Naive assumption - we assume that restrictions are never lifted, so the heaviest restriction reported for fips is still active.

We will likely have to revisit this in the future.

In [0]:
policies_min = full_policies.groupby(by="fips").agg(func=max)
policies_min.pop("county_name");

In [0]:
with open("county_policies.json", "w") as outfile:
  outfile.write(policies_min.to_json(orient="index"))

In [0]:
with open("county_policies.csv", "w") as outfile:
  outfile.write(policies_min.to_csv())